<a href="https://colab.research.google.com/github/Unagi6/TCIA_Documentation_milestoneTWO/blob/main/TCIA_Documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disclaimer
Title: Reading file of TCIA Data and Organizeation

Association: Fredrick National Labratory for Cancer Resarch

Data: [Cancer Imaging Archive](https://www.cancerimagingarchive.net/)

The Master Template Used: [Templet](https://docs.google.com/spreadsheets/d/1OGT3yv3GzMILPLh5kT7DrUstwy5zcXb_J0SonYh40EQ/edit?gid=0#gid=0)

# Summarize
Collaborating on datasets often leads to confusion, inconsistencies, and corrupted data, making it difficult to read and use for research.

This Google Colab notebook provides a streamlined workflow for collecting data from The Cancer Imaging Archive (TCIA) and remapping it to comply with National Institutes of Health (NIH) standards. This process ensures data is clean, consistent, and ready for research.


# Instalation
'**import sys**' followed by '**!{sys.executable} -m pip install --upgrade -q tcia_utils**', is designed to install or upgrade a Python package named tcia_utils. Specifically, import sys gives access to system-specific functionalities.

The shell command '**!{sys.executable} -m pip install --upgrade -q tcia_utils**' leverages the current Python interpreter to run pip in module mode, which is also  being instructed to install the tcia_utils package. The --upgrade flag ensures that if the package is already present, it's updated to its latest version.

The -q keeps the installation process quiet by suppressing most output.

In [1]:
import sys

# install tcia utils
!{sys.executable} -m pip install --upgrade -q tcia_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.7 MB/s eta 0:00:00


In [2]:
import requests
import datetime
import os
import pandas as pd
import subprocess
from tcia_utils import nbia
from tcia_utils import datacite
from tcia_utils import pathdb
from tcia_utils import wordpress
import plotly.express as px
import altair as alt

What to modify

# Note:
The notebooks should begin by importing the original spreadsheets from the TCIA collection of data. Please note that importing the spreadsheets directly from TCIA website make sure to right click on the download button of any clinical data and press '**copy link address**' to input into the panda reader.  The point is to enable someone to replicate your conversion without having to manually do anything outside of running the notebook.

In [3]:
# IMPORTANT: Always verify this URL directly from the TCIA UCSF-PDGM collection page
# (https://www.cancerimagingarchive.net/collection/ucsf-pdgm/)
# as TCIA may update direct download links.
# The search results indicate a clinical data file like this:
ucsf_pdgm_clinical_data_url = "https://www.cancerimagingarchive.net/wp-content/uploads/UCSF-PDGM-metadata_v5.csv"

print(f"Attempting to download clinical data from: {ucsf_pdgm_clinical_data_url}")

Attempting to download clinical data from: https://www.cancerimagingarchive.net/wp-content/uploads/UCSF-PDGM-metadata_v5.csv


# Data:
Data: UCSF-PDGM | The University of California San Francisco Preoperative Diffuse Glioma MRI

# What to do first:
When gettign the data from TCIA check to see if the clinical data can be read by pd

In [4]:
# Read the CSV file into a Pandas DataFrame
UCSF_data = pd.read_csv(ucsf_pdgm_clinical_data_url)

In [5]:
UCSF_data.head()

,ID,Sex,Age at MRI,WHO CNS Grade,Final pathologic diagnosis (WHO 2021),MGMT status,MGMT index,1p/19q,IDH,1-dead 0-alive,OS,EOR,Biopsy prior to imaging,BraTS21 ID,BraTS21 Segmentation Cohort,BraTS21 MGMT Cohort
0,UCSF-PDGM-004,M,66,4,"Glioblastoma, IDH-wildtype",negative,0,unknown,wildtype,1,1303.0,STR,No,BraTS2021_00097,Training,Training
1,UCSF-PDGM-005,F,80,4,"Glioblastoma, IDH-wildtype",indeterminate,unknown,unknown,wildtype,1,274.0,biopsy,No,NaN,NaN,NaN
2,UCSF-PDGM-007,M,70,4,"Glioblastoma, IDH-wildtype",indeterminate,unknown,unknown,wildtype,1,417.0,STR,No,BraTS2021_00103,Training,NaN
3,UCSF-PDGM-008,M,70,4,"Glioblastoma, IDH-wildtype",negative,0,unknown,wildtype,1,185.0,STR,No,NaN,NaN,NaN
4,UCSF-PDGM-009,F,68,4,"Glioblastoma, IDH-wildtype",negative,0,unknown,wildtype,1,389.0,STR,No,BraTS2021_00049,Training,Training


Convert the name of each relevant column header to our "Preferred Name".

Link: https://docs.google.com/spreadsheets/d/1OGT3yv3GzMILPLh5kT7DrUstwy5zcXb_J0SonYh40EQ/edit?gid=0#gid=0

If any column dosn't fit within the NIH standard of naming, it may either be '**drop**' or unmodified to the choice of the user.

In [6]:
ucsf_list = UCSF_data.columns
print(ucsf_list)

Index(['ID', 'Sex', 'Age at MRI', 'WHO CNS Grade',
       'Final pathologic diagnosis (WHO 2021)', 'MGMT status', 'MGMT index',
       '1p/19q', 'IDH', '1-dead 0-alive', 'OS', 'EOR',
       'Biopsy prior to imaging', 'BraTS21 ID', 'BraTS21 Segmentation Cohort',
       'BraTS21 MGMT Cohort'],
      dtype='object')


Lets rename all the columns to proper NIH prefered names.

In [7]:
# Using the .rename() method
df = UCSF_data.rename(columns={'ID': 'Subject ID', 'Sex': 'Sex at Birth',
                                'Age at MRI': 'Age at Imaging',
                                'Final pathologic diagnosis (WHO 2021)': 'Primary Diagnosis', # This is the new naming added on the doc.
                                '1-dead 0-alive': 'Vital Status' # 1-dead 0-alive
                                })
print("\nDataFrame after renaming:")



DataFrame after renaming:


Some Columns will have names and values set for the data such Column 'Vital Statues' where the data such as '1' being designatied as 'Dead' and '0' being designated as 'Alive'.

Now lets use mapping to change the data

1.   1 = Dead
2.   0 = Alive
3.   The rest are to be 'Not Recorded'

To maintain data integrity and logical consistency, any value falling outside of the defined '1' or '0' categories is designated as 'Not Recorded'.

In [8]:
vital_status_map = {
    0 : 'Alive',
    1: 'Dead'
}

df['Vital Status'] = df['Vital Status'].map(vital_status_map)
df['Vital Status'] = df['Vital Status'].fillna('Not Recorded') # If 1 or 0 does not appear, Not Recorded will be in place


In [9]:
# Getting rid of the .0 in OS
df['OS'] = df['OS'].astype(str) # There are string values with no .0
df['OS'] = df['OS'].str.replace('.0', '')

Following NIH naming conventions, a 'Project Short Name' must be included. For this dataset, the short name is UCSF-PDGM. This serves as a unique project identifier for each entry, similar to how a 'Subject ID' uniquely identifies each participant in the dataset.

For example, in a collection named "CPTAC-LUAD | The Clinical Proteomic Tumor Analysis Consortium Lung Adenocarcinoma Collection," the corresponding 'Project Short Name' is CPTAC-LUAD.

In [10]:
df['Project Short Name'] = 'UCSF-PDGM'

# Now we move the 'Project Short Name' infront of 'Subject ID'
cols = list(df.columns)
cols.insert(0, cols.pop(cols.index('Project Short Name')))
df = df.loc[:, cols]

The next step is to transform the values within your dataset. You'll replace non-standard abbreviations with their full, permissible values. For example, in the 'Sex at Birth' column, you will change "M" to "Male" and "F" to "Female". This ensures the data aligns with the detailed data report's specifications, a crucial step for achieving standardization.

In [11]:
df['Sex at Birth'] = df['Sex at Birth'].replace({'M': 'Male', 'F': 'Female'})
df.head()

,Project Short Name,Subject ID,Sex at Birth,Age at Imaging,WHO CNS Grade,Primary Diagnosis,MGMT status,MGMT index,1p/19q,IDH,Vital Status,OS,EOR,Biopsy prior to imaging,BraTS21 ID,BraTS21 Segmentation Cohort,BraTS21 MGMT Cohort
0,UCSF-PDGM,UCSF-PDGM-004,Male,66,4,"Glioblastoma, IDH-wildtype",negative,0,unknown,wildtype,Dead,1303,STR,No,BraTS2021_00097,Training,Training
1,UCSF-PDGM,UCSF-PDGM-005,Female,80,4,"Glioblastoma, IDH-wildtype",indeterminate,unknown,unknown,wildtype,Dead,274,biopsy,No,NaN,NaN,NaN
2,UCSF-PDGM,UCSF-PDGM-007,Male,70,4,"Glioblastoma, IDH-wildtype",indeterminate,unknown,unknown,wildtype,Dead,417,STR,No,BraTS2021_00103,Training,NaN
3,UCSF-PDGM,UCSF-PDGM-008,Male,70,4,"Glioblastoma, IDH-wildtype",negative,0,unknown,wildtype,Dead,185,STR,No,NaN,NaN,NaN
4,UCSF-PDGM,UCSF-PDGM-009,Female,68,4,"Glioblastoma, IDH-wildtype",negative,0,unknown,wildtype,Dead,389,STR,No,BraTS2021_00049,Training,Training


To complete the data transformation, you'll drop the columns that do not align with NIH standards. This step is crucial for ensuring the dataset is clean and consistent, containing only the information that is relevant and properly defined by the standard.

In [12]:
df = df.drop(columns=['MGMT status', 'MGMT index', '1p/19q', 'IDH', 'OS', 'EOR',
                 'Biopsy prior to imaging', 'BraTS21 ID',
                 'BraTS21 Segmentation Cohort', 'BraTS21 MGMT Cohort', 'WHO CNS Grade'])

# Display the first few rows of the modified DataFrame to verify the columns are gone
df.head()

,Project Short Name,Subject ID,Sex at Birth,Age at Imaging,Primary Diagnosis,Vital Status
0,UCSF-PDGM,UCSF-PDGM-004,Male,66,"Glioblastoma, IDH-wildtype",Dead
1,UCSF-PDGM,UCSF-PDGM-005,Female,80,"Glioblastoma, IDH-wildtype",Dead
2,UCSF-PDGM,UCSF-PDGM-007,Male,70,"Glioblastoma, IDH-wildtype",Dead
3,UCSF-PDGM,UCSF-PDGM-008,Male,70,"Glioblastoma, IDH-wildtype",Dead
4,UCSF-PDGM,UCSF-PDGM-009,Female,68,"Glioblastoma, IDH-wildtype",Dead


Congratulations on successfully harmonizing this dataset! You've successfully implemented all the necessary data validators and standardized the column headers. Now that you've mastered the process, the next step is to apply these same techniques to the other datasets, bringing them all up to the NIH's high standards.

Link being used: https://www.cancerimagingarchive.net/wp-content/uploads/Final-patient_list.xlsx

Data: Ovarian Bevacizumab Response

In [13]:
obr_df = pd.read_excel('https://www.cancerimagingarchive.net/wp-content/uploads/Final-patient_list.xlsx')
obr_df.head()

,No.,Patient ID,Age,Diagnosis,FIGO stage,Unnamed: 5,operation,method for avastin use,number of avastin administration,operation date,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1767721.0,70.0,UC (unclassified carcinoma),III,invalid,Suboptimal debulking,2nd line,3.0,2005-10-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,1851333.0,60.0,PsC (Papillary serous carcinoma),IV,invalid,optimal debulking,2nd line,6.0,2005-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,2909711.0,63.0,PSPC (Peritoneal serous papillary carcinoma),III,invalid,CRS+HIPEC,front line,6.0,2007-11-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,2004960.0,72.0,PsC,III,invalid,Suboptimal debulking,2nd line,4.0,2001-10-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Find out the columns:

In [14]:
obr_list = obr_df.columns
print(obr_list)

Index([                              'No.',
                              'Patient ID',
                                     'Age',
                               'Diagnosis',
                              'FIGO stage',
                              'Unnamed: 5',
                               'operation',
                  'method for avastin use',
        'number of avastin administration',
                          'operation date',
       'starting date for use of avastin ',
             'End date for use of avastin',
                          'recurrent date',
                           'Date of death',
                                     'BMI',
                             'Unnamed: 15',
                             'Unnamed: 16',
                             'Unnamed: 17',
                             'Unnamed: 18',
                             'Unnamed: 19',
                             'Unnamed: 20',
                             'Unnamed: 21',
                             'Un

Time to reomve 'Unnamed: 15' all the way to '1' along with 'No.' and 'Unnamed: 5'. Also removing row zero.

In [15]:
df1 = obr_df.drop(columns=['No.', 'Unnamed: 15', 'Unnamed: 5', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
                           'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23','Unnamed: 24',
                           'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
                           'Unnamed: 30', 1])
df1 = df1.drop(0)

Change the Diagnosis naming data.

In [16]:
# Accessing the 'Diagnosis' column
diagnosis_column = df1['Diagnosis']
print("\nDiagnosis data (as a Pandas Series):\n", diagnosis_column)


Diagnosis data (as a Pandas Series):
 1                      UC (unclassified carcinoma)
2                 PsC (Papillary serous carcinoma)
3     PSPC (Peritoneal serous papillary carcinoma)
4                                              PsC
5                                              PsC
                          ...                     
74                                             PsC
75                                            PSPC
76                                              UC
77                                              UC
78                                             PsC
Name: Diagnosis, Length: 78, dtype: object


Changing the Data to proper NIH names.

In [17]:
diagnosis_map = {
    'UC (unclassified carcinoma)': 'Unclassified Carcinoma',
    'UC': 'Unclassified Carcinoma',
    'PsC (Papillary serous carcinoma)': 'Papillary Serous Carcinoma',
    'PsC': 'Papillary Carcinoma',
    'PSPC (Peritoneal serous papillary carcinoma)': 'Papillary Serous Carcinoma',
    'PSPC': 'Papillary Squamous Carcinoma'
}

df1['Diagnosis'] = df1['Diagnosis'].map(diagnosis_map)
df1['Diagnosis'] = df1['Diagnosis'].fillna('Not Recorded') # If 1 or 0 does not appear, Not Recorded will be in place

# Turning Patient ID, Age at Imaging, and number of avastin administration
df1['Patient ID'] = df1['Patient ID'].astype(int)
df1['Age'] = df1['Age'].astype(int)
df1['number of avastin administration'] = df1['number of avastin administration'].astype(int)

Renaming Columns to NIH standard

In [18]:
df1 = df1.rename(columns={'Patient ID' : 'Subject ID', 'Age': 'Age at Imaging', 'Diagnosis': 'Primary Diagnosis',
                          'Date of death': 'Age at Death'})
df1.head()

,Subject ID,Age at Imaging,Primary Diagnosis,FIGO stage,operation,method for avastin use,number of avastin administration,operation date,starting date for use of avastin,End date for use of avastin,recurrent date,Age at Death,BMI
1,1767721,70,Unclassified Carcinoma,III,Suboptimal debulking,2nd line,3,2005-10-26,2006-08-13,2006-10-10,2006-10-26 00:00:00,survival,21.2
2,1851333,60,Papillary Serous Carcinoma,IV,optimal debulking,2nd line,6,2005-10-01,2007-03-28,2007-07-19,2007-08-26 00:00:00,2007-10-10 00:00:00,20.3
3,2909711,63,Papillary Serous Carcinoma,III,CRS+HIPEC,front line,6,2007-11-13,2007-11-20,2008-04-04,2008-04-04 00:00:00,survival,25.2
4,2004960,72,Papillary Carcinoma,III,Suboptimal debulking,2nd line,4,2001-10-13,2006-05-06,2006-10-07,2006-07-22 00:00:00,2007-03-19 00:00:00,21.4
5,3014307,56,Papillary Carcinoma,II,optimal debulking,front line,6,2009-04-21,2009-05-20,2009-08-22,no recurrence,survival,27.1


Changes to the code: Sublime - Have the app read the first row, if empty, use second row as column data. The headers are in row.

Data: COVID-19-AR | Chest Imaging with Clinical and Genomic Correlates Representing a Rural COVID-19 Positive Population

In [19]:
covid_df = pd.read_excel('https://www.cancerimagingarchive.net/wp-content/uploads/COVID-19-AR-Clinical-Correlates-July202020.xlsx')
covid_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,PATIENT_ID,AGE,SEX,RACE,ZIP,LATEST_BMI,LATEST WEIGHT,LATEST HEIGHT,TUBERCULOSIS,SYSTEMIC LUPUS ERYTHMATOSUS,...,DIABETES TYPE II,TRANSPLANT,HEMODIALYSIS Pre Diagnosis,HEMODIALYSIS Post diagnosis,CANCER,COVID TEST POSITIVE,TEST NAME,ICU Admit,# ICU admits,MORTALITY
1,COVID-19-AR-16434409,51,M,BLACK OR AFRICAN AMERICAN,722,37.7,207,"5' 4""",N,N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N
2,COVID-19-AR-16434381,44,F,BLACK OR AFRICAN AMERICAN,721,64.6,412,"5' 7""",N,N,...,N,N,N,N,N,Y,NaN,N,0,N
3,COVID-19-AR-16406513,44,M,BLACK OR AFRICAN AMERICAN,722,33.3,190,"5' 5""",N,N,...,N,N,N,N,N,Y,Misc Test: covid-19,Y,1,N
4,COVID-19-AR-16439216,61,F,BLACK OR AFRICAN AMERICAN,722,36,210,"5' 4""",N,N,...,Y,N,N,N,Y,Y,NaN,N,0,N


First make row 0 the new column.

In [20]:
covid_df.columns = covid_df.iloc[0]
covid_df = covid_df[1:]

In [21]:
covid_df = covid_df.rename(columns={'PATIENT_ID': 'Subject ID', 'SEX': 'Sex at Birth'})
covid_df['Sex at Birth'] = covid_df['Sex at Birth'].replace({'M': 'Male', 'F': 'Female'})
covid_df.head()

,Subject ID,AGE,Sex at Birth,RACE,ZIP,LATEST_BMI,LATEST WEIGHT,LATEST HEIGHT,TUBERCULOSIS,SYSTEMIC LUPUS ERYTHMATOSUS,...,DIABETES TYPE II,TRANSPLANT,HEMODIALYSIS Pre Diagnosis,HEMODIALYSIS Post diagnosis,CANCER,COVID TEST POSITIVE,TEST NAME,ICU Admit,# ICU admits,MORTALITY
1,COVID-19-AR-16434409,51,Male,BLACK OR AFRICAN AMERICAN,722,37.7,207,"5' 4""",N,N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N
2,COVID-19-AR-16434381,44,Female,BLACK OR AFRICAN AMERICAN,721,64.6,412,"5' 7""",N,N,...,N,N,N,N,N,Y,NaN,N,0,N
3,COVID-19-AR-16406513,44,Male,BLACK OR AFRICAN AMERICAN,722,33.3,190,"5' 5""",N,N,...,N,N,N,N,N,Y,Misc Test: covid-19,Y,1,N
4,COVID-19-AR-16439216,61,Female,BLACK OR AFRICAN AMERICAN,722,36,210,"5' 4""",N,N,...,Y,N,N,N,Y,Y,NaN,N,0,N
5,COVID-19-AR-16406491,49,Female,BLACK OR AFRICAN AMERICAN,721,43.85,280,"5' 7""",N,N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N


In [22]:
covid_df['Project Short Name'] = 'COVID-19-AR'

# Move the Short Name to the front
cols = list(covid_df.columns)
cols.insert(0, cols.pop(cols.index('Project Short Name')))
covid_df = covid_df[cols]
covid_df.head()

,Project Short Name,Subject ID,AGE,Sex at Birth,RACE,ZIP,LATEST_BMI,LATEST WEIGHT,LATEST HEIGHT,TUBERCULOSIS,...,DIABETES TYPE II,TRANSPLANT,HEMODIALYSIS Pre Diagnosis,HEMODIALYSIS Post diagnosis,CANCER,COVID TEST POSITIVE,TEST NAME,ICU Admit,# ICU admits,MORTALITY
1,COVID-19-AR,COVID-19-AR-16434409,51,Male,BLACK OR AFRICAN AMERICAN,722,37.7,207,"5' 4""",N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N
2,COVID-19-AR,COVID-19-AR-16434381,44,Female,BLACK OR AFRICAN AMERICAN,721,64.6,412,"5' 7""",N,...,N,N,N,N,N,Y,NaN,N,0,N
3,COVID-19-AR,COVID-19-AR-16406513,44,Male,BLACK OR AFRICAN AMERICAN,722,33.3,190,"5' 5""",N,...,N,N,N,N,N,Y,Misc Test: covid-19,Y,1,N
4,COVID-19-AR,COVID-19-AR-16439216,61,Female,BLACK OR AFRICAN AMERICAN,722,36,210,"5' 4""",N,...,Y,N,N,N,Y,Y,NaN,N,0,N
5,COVID-19-AR,COVID-19-AR-16406491,49,Female,BLACK OR AFRICAN AMERICAN,721,43.85,280,"5' 7""",N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N


Data: COVID-19-NY-SBU | Stony Brook University COVID-19 Positive Cases

In [23]:
ny_df = pd.read_excel('https://www.cancerimagingarchive.net/wp-content/uploads/COVID-19-AR-Clinical-Correlates-July202020.xlsx')
ny_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,PATIENT_ID,AGE,SEX,RACE,ZIP,LATEST_BMI,LATEST WEIGHT,LATEST HEIGHT,TUBERCULOSIS,SYSTEMIC LUPUS ERYTHMATOSUS,...,DIABETES TYPE II,TRANSPLANT,HEMODIALYSIS Pre Diagnosis,HEMODIALYSIS Post diagnosis,CANCER,COVID TEST POSITIVE,TEST NAME,ICU Admit,# ICU admits,MORTALITY
1,COVID-19-AR-16434409,51,M,BLACK OR AFRICAN AMERICAN,722,37.7,207,"5' 4""",N,N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N
2,COVID-19-AR-16434381,44,F,BLACK OR AFRICAN AMERICAN,721,64.6,412,"5' 7""",N,N,...,N,N,N,N,N,Y,NaN,N,0,N
3,COVID-19-AR-16406513,44,M,BLACK OR AFRICAN AMERICAN,722,33.3,190,"5' 5""",N,N,...,N,N,N,N,N,Y,Misc Test: covid-19,Y,1,N
4,COVID-19-AR-16439216,61,F,BLACK OR AFRICAN AMERICAN,722,36,210,"5' 4""",N,N,...,Y,N,N,N,Y,Y,NaN,N,0,N


Like the AR covid data set, NY covid dataset has the same problems

In [24]:
# Make row 0 the new column
ny_df.columns = ny_df.iloc[0]
ny_df = ny_df[1:]

NOTE: DO NOT COMBINE ny_df.head with above code due to destroying the dataset by getting rid of the row.

In [25]:
ny_df.head()

,PATIENT_ID,AGE,SEX,RACE,ZIP,LATEST_BMI,LATEST WEIGHT,LATEST HEIGHT,TUBERCULOSIS,SYSTEMIC LUPUS ERYTHMATOSUS,...,DIABETES TYPE II,TRANSPLANT,HEMODIALYSIS Pre Diagnosis,HEMODIALYSIS Post diagnosis,CANCER,COVID TEST POSITIVE,TEST NAME,ICU Admit,# ICU admits,MORTALITY
1,COVID-19-AR-16434409,51,M,BLACK OR AFRICAN AMERICAN,722,37.7,207,"5' 4""",N,N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N
2,COVID-19-AR-16434381,44,F,BLACK OR AFRICAN AMERICAN,721,64.6,412,"5' 7""",N,N,...,N,N,N,N,N,Y,NaN,N,0,N
3,COVID-19-AR-16406513,44,M,BLACK OR AFRICAN AMERICAN,722,33.3,190,"5' 5""",N,N,...,N,N,N,N,N,Y,Misc Test: covid-19,Y,1,N
4,COVID-19-AR-16439216,61,F,BLACK OR AFRICAN AMERICAN,722,36,210,"5' 4""",N,N,...,Y,N,N,N,Y,Y,NaN,N,0,N
5,COVID-19-AR-16406491,49,F,BLACK OR AFRICAN AMERICAN,721,43.85,280,"5' 7""",N,N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N


In [26]:
ny_df = ny_df.rename(columns={'PATIENT_ID': 'Subject ID', 'SEX': 'Sex at Birth'})
ny_df['Sex at Birth'] = ny_df['Sex at Birth'].replace({'M': 'Male', 'F': 'Female'})

# Create Project name
ny_df['Project Short Name'] = 'COVID-19-NY-SBU'

# Move the Short Name to the front
cols1 = list(ny_df.columns)
cols1.insert(0, cols1.pop(cols1.index('Project Short Name')))
ny_df = ny_df[cols1]
ny_df.head()

,Project Short Name,Subject ID,AGE,Sex at Birth,RACE,ZIP,LATEST_BMI,LATEST WEIGHT,LATEST HEIGHT,TUBERCULOSIS,...,DIABETES TYPE II,TRANSPLANT,HEMODIALYSIS Pre Diagnosis,HEMODIALYSIS Post diagnosis,CANCER,COVID TEST POSITIVE,TEST NAME,ICU Admit,# ICU admits,MORTALITY
1,COVID-19-NY-SBU,COVID-19-AR-16434409,51,Male,BLACK OR AFRICAN AMERICAN,722,37.7,207,"5' 4""",N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N
2,COVID-19-NY-SBU,COVID-19-AR-16434381,44,Female,BLACK OR AFRICAN AMERICAN,721,64.6,412,"5' 7""",N,...,N,N,N,N,N,Y,NaN,N,0,N
3,COVID-19-NY-SBU,COVID-19-AR-16406513,44,Male,BLACK OR AFRICAN AMERICAN,722,33.3,190,"5' 5""",N,...,N,N,N,N,N,Y,Misc Test: covid-19,Y,1,N
4,COVID-19-NY-SBU,COVID-19-AR-16439216,61,Female,BLACK OR AFRICAN AMERICAN,722,36,210,"5' 4""",N,...,Y,N,N,N,Y,Y,NaN,N,0,N
5,COVID-19-NY-SBU,COVID-19-AR-16406491,49,Female,BLACK OR AFRICAN AMERICAN,721,43.85,280,"5' 7""",N,...,Y,N,N,N,N,Y,SARS-CoV-2 (COVID-19),N,0,N


Data: CRC_FFPE-CODEX_CellNeighs | High-dimensional imaging of colorectal carcinoma and other tumors with 50+ markers

In [27]:
crc_df = pd.read_excel('https://www.cancerimagingarchive.net/wp-content/uploads/CRC_TMAs_patient_annotations.xlsx')
crc_df.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Patient,Group,TMA spot / region,LA,Diffuse,LA.1,Diffuse.1,Klintrup_Makinen,CLR_Graham_Appelman,Sex,...,DFS,DFS_Censor,Nbr_Buds_ITBCC,Budding grade,MLH1,PMS2,MSH6,MSH2,MSI_IHC,MSI_PCR
0,1.0,1,"1,2",1,1.0,1,1.0,2.0,2.0,Female,...,71.334648,0.0,25.0,3.0,1,1,1,1,MSS,0.0
1,2.0,2,"3,4",NaN,2.0,NaN,2.0,2.0,0.0,Female,...,137.639711,1.0,8.0,2.0,1,1,1,1,MSS,0.0
2,3.0,2,"5,6",NaN,2.0,NaN,2.0,2.0,0.0,Male,...,0.394477,0.0,5.0,2.0,1,1,1,1,MSS,0.0
3,4.0,2,"7,8",NaN,2.0,NaN,2.0,2.0,0.0,Male,...,0.525970,0.0,0.0,1.0,1,0,1,1,MSI,0.0
4,5.0,2,"9,10",NaN,2.0,NaN,2.0,2.0,0.0,Female,...,43.458251,0.0,12.0,3.0,1,1,1,1,MSS,0.0


In [28]:
# Rename patient with Subject ID
crc_df = crc_df.rename(columns={'Patient': 'Subject ID'})

# Add the Project Short Name
crc_df['Project Short Name'] = 'CRC_FFPE-CODEX_CellNeighs'
# Move the Short Name to the front
cols2 = list(crc_df.columns)
cols2.insert(0, cols2.pop(cols2.index('Project Short Name')))
crc_df = crc_df[cols2]
crc_df.head()

,Project Short Name,Subject ID,Group,TMA spot / region,LA,Diffuse,LA.1,Diffuse.1,Klintrup_Makinen,CLR_Graham_Appelman,...,DFS,DFS_Censor,Nbr_Buds_ITBCC,Budding grade,MLH1,PMS2,MSH6,MSH2,MSI_IHC,MSI_PCR
0,CRC_FFPE-CODEX_CellNeighs,1.0,1,"1,2",1,1.0,1,1.0,2.0,2.0,...,71.334648,0.0,25.0,3.0,1,1,1,1,MSS,0.0
1,CRC_FFPE-CODEX_CellNeighs,2.0,2,"3,4",NaN,2.0,NaN,2.0,2.0,0.0,...,137.639711,1.0,8.0,2.0,1,1,1,1,MSS,0.0
2,CRC_FFPE-CODEX_CellNeighs,3.0,2,"5,6",NaN,2.0,NaN,2.0,2.0,0.0,...,0.394477,0.0,5.0,2.0,1,1,1,1,MSS,0.0
3,CRC_FFPE-CODEX_CellNeighs,4.0,2,"7,8",NaN,2.0,NaN,2.0,2.0,0.0,...,0.525970,0.0,0.0,1.0,1,0,1,1,MSI,0.0
4,CRC_FFPE-CODEX_CellNeighs,5.0,2,"9,10",NaN,2.0,NaN,2.0,2.0,0.0,...,43.458251,0.0,12.0,3.0,1,1,1,1,MSS,0.0


Data: HER2 tumor ROIs | HER2 and trastuzumab treatment response H&E slides with tumor ROI annotations

Note: File might be missing from link or currupted. https://www.cancerimagingarchive.net/wp-content/uploads/Yale_trastuzumab_response_cohort_metadata_clean.xlsx

In [29]:
#df4 = pd.read_excel('https://www.cancerimagingarchive.net/wp-content/uploads/Yale_trastuzumab_response_cohort_metadata_clean.xlsx')
#df4.head()

In [30]:
import pandas as pd
import requests
from io import BytesIO

# The URL of your Excel file
excel_url = 'https://www.cancerimagingarchive.net/wp-content/uploads/Yale_trastuzumab_response_cohort_metadata_clean.xlsx'

try:
    # Send a GET request to the URL to fetch the file content
    response = requests.get(excel_url)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

    # Use BytesIO to treat the content as a file-like object
    excel_file_content = BytesIO(response.content)

    # Read the Excel file into a pandas DataFrame
    # You might need to specify the sheet name or index if it's not the first sheet,
    # but the error implies no sheets were found at all.
    # If there's only one sheet, pandas often defaults correctly.
    her2_df = pd.read_excel(excel_file_content)

    # Display the first few rows of the DataFrame to verify
    her2_df.head()

except requests.exceptions.RequestException as e:
    print(f"Error fetching the file from URL: {e}")
except ValueError as e:
    print(f"Error reading the Excel file with pandas: {e}. This might indicate the file structure is not as expected or the file is corrupted.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error reading the Excel file with pandas: Worksheet index 0 is invalid, 0 worksheets found. This might indicate the file structure is not as expected or the file is corrupted.


/usr/local/lib/python3.11/dist-packages/openpyxl/reader/workbook.py:84: UserWarning: File contains an invalid specification for 0. This will be removed
  warn(msg)


Data: Lung-PET-CT-Dx | A Large-Scale CT and PET/CT Dataset for Lung Cancer Diagnosis

In [31]:
lung_df = pd.read_excel('https://www.cancerimagingarchive.net/wp-content/uploads/statistics-clinical-20201221.xlsx')
lung_df.head()

,No.,NewPatientID,Sex,Age,weight (kg),T-Stage,N-Stage,Ｍ-Stage,Histopathological grading,Smoking History
0,1,A0001,M,58.0,65.0,2b,3,1b,G3,1
1,2,A0002,F,53.0,55.0,2b,1,0,NaN,0
2,3,A0003,M,60.0,62.0,1c,1,0,G3,0
3,4,A0004,F,48.0,56.0,1c,3,0,NaN,0
4,5,A0005,M,70.0,75.0,1b,0,0,G2,1


In [32]:
# Rename newpatientid to subject id, Sex to Sex at Birth, and Age to Age at Imaging
lung_df = lung_df.rename(columns={'NewPatientID': 'Subject ID', 'Sex': 'Sex at Birth', 'Age': 'Age at Imaging', 'Ｍ-Stage': 'M-Stage'})

# Add the Project Short Name
lung_df['Project Short Name'] = 'Lung-PET-CT-Dx'
# Move new column to the front
cols3 = list(lung_df.columns)
cols3.insert(0, cols3.pop(cols3.index('Project Short Name')))
lung_df = lung_df[cols3]


# Replace M and F with Male and Female
lung_df['Sex at Birth'] = lung_df['Sex at Birth'].replace({'M': 'Male', 'F': 'Female'})

# Get rid of .0 in age at imaging
lung_df['Age at Imaging'] = lung_df['Age at Imaging'].astype(str)
lung_df['Age at Imaging'] = lung_df['Age at Imaging'].str.replace('.0', '')

#put T infront of the data in T-Stage
lung_df['T-Stage'] = lung_df['T-Stage'].astype(str)
lung_df['T-Stage'] = 'T' + lung_df['T-Stage']

#same to N
lung_df['N-Stage'] = lung_df['N-Stage'].astype(str)
lung_df['N-Stage'] = 'N' + lung_df['N-Stage']

#same to M
lung_df['M-Stage'] = lung_df['M-Stage'].astype(str)
lung_df['M-Stage'] = 'M' + lung_df['M-Stage']

lung_df.head()

,Project Short Name,No.,Subject ID,Sex at Birth,Age at Imaging,weight (kg),T-Stage,N-Stage,M-Stage,Histopathological grading,Smoking History
0,Lung-PET-CT-Dx,1,A0001,Male,58,65.0,T2b,N3,M1b,G3,1
1,Lung-PET-CT-Dx,2,A0002,Female,53,55.0,T2b,N1,M0,NaN,0
2,Lung-PET-CT-Dx,3,A0003,Male,60,62.0,T1c,N1,M0,G3,0
3,Lung-PET-CT-Dx,4,A0004,Female,48,56.0,T1c,N3,M0,NaN,0
4,Lung-PET-CT-Dx,5,A0005,Male,70,75.0,T1b,N0,M0,G2,1


Data: NSCLC Radiogenomics | NSCLC Radiogenomics

In [43]:
df5 = pd.read_csv('https://www.cancerimagingarchive.net/wp-content/uploads/NSCLCR01Radiogenomic_DATA_LABELS_2018-05-22_1500-shifted.csv')
df5.head()

,Case ID,Patient affiliation,Age at Histological Diagnosis,Weight (lbs),Gender,Ethnicity,Smoking status,Pack Years,Quit Smoking Year,%GG,...,Recurrence,Recurrence Location,Date of Recurrence,Date of Last Known Alive,Survival Status,Date of Death,Time to Death (days),CT Date,Days between CT and surgery,PET Date
0,AMC-001,Stanford,34,Not Collected,Male,Not Recorded In Database,Nonsmoker,NaN,NaN,Not Assessed,...,yes,distant,10/7/1994,1/7/1997,Dead,1/7/1997,872.0,8/10/1994,9,Not Collected
1,AMC-002,Stanford,33,Not Collected,Female,Not Recorded In Database,Nonsmoker,NaN,NaN,Not Assessed,...,no,NaN,NaN,3/20/1992,Alive,NaN,NaN,2/19/1992,3,Not Collected
2,AMC-003,Stanford,69,Not Collected,Female,Not Recorded In Database,Nonsmoker,NaN,NaN,Not Assessed,...,no,NaN,NaN,6/19/1996,Alive,NaN,NaN,2/23/1995,28,Not Collected
3,AMC-004,Stanford,80,Not Collected,Female,Not Recorded In Database,Nonsmoker,NaN,NaN,Not Assessed,...,no,NaN,NaN,12/13/1996,Alive,NaN,NaN,12/26/1992,47,Not Collected
4,AMC-005,Stanford,76,Not Collected,Male,Not Recorded In Database,Former,30,1962.0,Not Assessed,...,yes,distant,1/4/1996,1/7/1997,Alive,NaN,NaN,7/21/1994,2,Not Collected


In [44]:
# Add the Project Short Name
df5['Project Short Name'] = 'NSCLC Radiogenomics'
# Move new column to the front
cols4 = list(df5.columns)
cols4.insert(0, cols4.pop(cols4.index('Project Short Name')))
df5 = df5[cols4]

# Rename Case ID to Subject ID
df5 = df5.rename(columns={'Case ID': 'Subject ID', 'Gender': 'Sex at Birth', 'Survival Status': 'Vital Status',
                          'Date of Last Known Alive': 'Age at Death', 'Age at Histological Diagnosis': 'Age at Diagnosis'})

df5.head()

,Project Short Name,Subject ID,Patient affiliation,Age at Diagnosis,Weight (lbs),Sex at Birth,Ethnicity,Smoking status,Pack Years,Quit Smoking Year,...,Recurrence,Recurrence Location,Date of Recurrence,Age at Death,Vital Status,Date of Death,Time to Death (days),CT Date,Days between CT and surgery,PET Date
0,NSCLC Radiogenomics,AMC-001,Stanford,34,Not Collected,Male,Not Recorded In Database,Nonsmoker,NaN,NaN,...,yes,distant,10/7/1994,1/7/1997,Dead,1/7/1997,872.0,8/10/1994,9,Not Collected
1,NSCLC Radiogenomics,AMC-002,Stanford,33,Not Collected,Female,Not Recorded In Database,Nonsmoker,NaN,NaN,...,no,NaN,NaN,3/20/1992,Alive,NaN,NaN,2/19/1992,3,Not Collected
2,NSCLC Radiogenomics,AMC-003,Stanford,69,Not Collected,Female,Not Recorded In Database,Nonsmoker,NaN,NaN,...,no,NaN,NaN,6/19/1996,Alive,NaN,NaN,2/23/1995,28,Not Collected
3,NSCLC Radiogenomics,AMC-004,Stanford,80,Not Collected,Female,Not Recorded In Database,Nonsmoker,NaN,NaN,...,no,NaN,NaN,12/13/1996,Alive,NaN,NaN,12/26/1992,47,Not Collected
4,NSCLC Radiogenomics,AMC-005,Stanford,76,Not Collected,Male,Not Recorded In Database,Former,30,1962.0,...,yes,distant,1/4/1996,1/7/1997,Alive,NaN,NaN,7/21/1994,2,Not Collected


Now for the final step is to drop the columns that does not fit the NIH standard.

While pandas' drop function makes column removal seem easy, datasets like NSCLC's can be so massive that they demand extensive dropping of numerous columns, making data preparation a considerable task.

In [46]:
df5 = df5.drop(columns=['Patient affiliation', 'Smoking status', 'Pack Years',
                        'Quit Smoking Year', 'Recurrence', 'Recurrence Location',
                        'Date of Recurrence', 'Days between CT and surgery',
                        'PET Date', '%GG', 'Tumor Location (choice=LUL)', 'Tumor Location (choice=LLL)',
                        'Tumor Location (choice=L Lingula)', 'Tumor Location (choice=Unknown)',
                        'KRAS mutation status', 'ALK translocation status',
                        'Adjuvant Treatment', 'Chemotherapy', 'Radiation', 'Date of Death', 'Time to Death (days)',
                        'Tumor Location (choice=RUL)', 'Tumor Location (choice=RML)', 'Tumor Location (choice=RLL)', 'EGFR mutation status',
                        'CT Date', 'Pleural invasion (elastic, visceral, or parietal)', 'Lymphovascular invasion', 'Histopathological Grade'])
df5.head()

,Project Short Name,Subject ID,Age at Diagnosis,Weight (lbs),Sex at Birth,Ethnicity,Histology,Pathological T stage,Pathological N stage,Pathological M stage,Age at Death,Vital Status
0,NSCLC Radiogenomics,AMC-001,34,Not Collected,Male,Not Recorded In Database,Adenocarcinoma,Not Collected,Not Collected,Not Collected,1/7/1997,Dead
1,NSCLC Radiogenomics,AMC-002,33,Not Collected,Female,Not Recorded In Database,Adenocarcinoma,Not Collected,Not Collected,Not Collected,3/20/1992,Alive
2,NSCLC Radiogenomics,AMC-003,69,Not Collected,Female,Not Recorded In Database,Adenocarcinoma,Not Collected,Not Collected,Not Collected,6/19/1996,Alive
3,NSCLC Radiogenomics,AMC-004,80,Not Collected,Female,Not Recorded In Database,Adenocarcinoma,Not Collected,Not Collected,Not Collected,12/13/1996,Alive
4,NSCLC Radiogenomics,AMC-005,76,Not Collected,Male,Not Recorded In Database,Adenocarcinoma,Not Collected,Not Collected,Not Collected,1/7/1997,Alive


# Conclusion:
Cleaning datasets can often be a challenging and complex process. This notebook aims to provide a comprehensive guide that will help you navigate these challenges by demonstrating how to effectively clean, standardize, and adhere to the National Institutes of Health naming conventions for both data fields and file names. By following these principles, you can ensure your data is consistent, interoperable, and ready for analysis and collaboration.